In [1]:
import pickle

from tweet_processor import TweetProcessor
from word_embedding_model import WordEmbeddingModel
from sentence_embedding_model import SentenceEmbeddingModel

dataset_file_path = 'datasets/hashtags-en-tweets.jsonl'

tweet_processor = TweetProcessor()

# word_emb_tweets = tweet_processor.preprocess_tweets_for_word_embedding(dataset_file_path)

# word_embedding_model = WordEmbeddingModel()
# word_embedding_model.train(word_emb_tweets)

# pickle.dump(word_embedding_model, open('save_files/word_emb_model.pkl', 'wb'))

word_emb_model = pickle.load(open('save_files/word_emb_model.pkl', 'rb'))

sent_emb_tweets = tweet_processor.preprocess_tweets_for_sentence_embedding(dataset_file_path)

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataset import DataModule
from hashtag_to_sent_mapper import Hashtag2SentMapper

import torch

data_module = DataModule.restore_from_file()
data_module.batch_size = 500
data_module.num_workers = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

hashtag_to_sent_mapper = Hashtag2SentMapper(data_module.in_features, data_module.out_features, learning_rate=0.001, hidden_layer1_size=300, hidden_layer2_size=500).to(device)

In [3]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import pytorch_lightning as pl

# torch.autograd.set_detect_anomaly(True)

trainer = pl.Trainer(
    gpus=0, max_epochs=500, 
    callbacks=[EarlyStopping(monitor='val_loss', patience=200)],
# callbacks=[EarlyStopping(monitor='val_cosine_distance', patience=200)],  
#  track_grad_norm=2, detect_anomaly=True, 
    log_every_n_steps=1
) 
trainer.fit(
    hashtag_to_sent_mapper, 
    data_module.train_dataloader(),
    data_module.val_dataloader()
)
trainer.test(dataloaders=[data_module.test_dataloader()])
trainer.save_checkpoint('save_files/best_model.ckpt', weights_only=True)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name                  | Type                | Params
--------------------------------------------------------------
0 | _model                | Sequential          | 580 K 
1 | _cosine_distance_loss | CosineEmbeddingLoss | 0     
2 | _cosine_similarity    | CosineSimilarity    | 0     
--------------------------------------------------------------
580 K     Trainable params
0         Non-trainable params
580 K     Total params
2.322     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 7 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


/usr/local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 7 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Epoch 499: 100%|██████████| 7/7 [00:00<00:00,  8.62it/s, loss=0.00881, v_num=20]

/usr/local/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1447: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `test(ckpt_path='best')` to use and best model checkpoint and avoid this warning or `ckpt_path=trainer.checkpoint_callback.last_model_path` to use the last model.
  f"`.{fn}(ckpt_path=None)` was called without a model."
Restoring states from the checkpoint path at /workspaces/HASHET/lightning_logs/version_20/checkpoints/epoch=499-step=3000.ckpt


Loaded model weights from checkpoint at /workspaces/HASHET/lightning_logs/version_20/checkpoints/epoch=499-step=3000.ckpt
/usr/local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 7 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 38.60it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
  test_cosine_distance      0.29528093338012695
        test_loss          0.022327067330479622
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
